In [1]:
%pip install jupyterlab jupyterlab-optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 31.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached async_lru-2.0.4-py3-none-any.whl (6.1 kB)
  Using cached httpx-0.27.0-py3-none-any.whl (75 kB)
  Using cached ipykernel-6.29.4-py3-none-any.whl (117 kB)
  Using cached jupyter_lsp-2.2.5-py3-none-any.whl (69 kB)
  Using cached jupyter_server-2.14.1-py3-none-any.whl (383 kB)
  Using cached jupyterlab_server-2.27.2-py3-none-any.whl (59 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 79.0 MB/s eta 0:00:00
  Using cached httpcore-1.0.5-py3-none-any.whl (77 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Using cached comm-0.2.2-py3-none-any.whl (7.2 kB)
  Using cached jupyter_client-8.6.2-py3-none-any.whl (105 kB)
  Using c

In [2]:
%pip install optuna-dashboard

In [10]:
import pandas as pd
import optuna
import lightgbm as lgb
!pip install lightgbm
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from optuna.visualization import plot_optimization_history

In [11]:
df = pd.read_csv("/content/Clean_train.csv")

In [12]:
x = df['text']
y = df['target']

In [13]:
tfidf_vectorizer = TfidfVectorizer()

In [14]:
x_tfidf = tfidf_vectorizer.fit_transform(x)

In [15]:
x_train, x_test, y_train, y_test = train_test_split(x_tfidf,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=42 ,
                                                    shuffle = True)

In [16]:
lgb = LGBMClassifier()

In [20]:
# Define the objective function for Optuna
def objective(trial):
    # Define the search space for LightGBM
    lgbm_params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'num_leaves': trial.suggest_int('num_leaves', 10, 100),
        # Add more hyperparameters to search here
    }

    # Train LightGBM model
    lgb_model = LGBMClassifier(**lgbm_params)
    lgb_model.fit(X_train, y_train)

    # Define the search space for logistic regression
    lr_params = {
        'C': trial.suggest_float('C', 0.1, 10.0),
        # Add more hyperparameters to search here
    }

    # Train logistic regression model
    lr_model = LogisticRegression(**lr_params)
    lr_model.fit(X_train, y_train)

    # Predictions
    lgb_pred = lgb_model.predict(X_valid)
    lr_pred = lr_model.predict(X_valid)

    # Compute accuracy
    lgb_accuracy = accuracy_score(y_valid, lgb_pred)
    lr_accuracy = accuracy_score(y_valid, lr_pred)

    # Return the performance as the objective value to maximize
    return lgb_accuracy + lr_accuracy


In [21]:
X_train, X_valid, y_train, y_valid = train_test_split(x_tfidf, y, test_size=0.2, random_state=42, shuffle=True)

In [22]:
# Set up Optuna study and optimize
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

[I 2024-06-28 11:28:06,853] A new study created in memory with name: no-name-53d50abd-f273-4432-ae41-6d7ea362a3e0


[LightGBM] [Info] Number of positive: 2622, number of negative: 3468
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.036693 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8984
[LightGBM] [Info] Number of data points in the train set: 6090, number of used features: 573
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.430542 -> initscore=-0.279641
[LightGBM] [Info] Start training from score -0.279641


[I 2024-06-28 11:28:09,274] Trial 0 finished with value: 1.571240971766251 and parameters: {'learning_rate': 0.06286821262065109, 'num_leaves': 57, 'C': 9.176007704988644}. Best is trial 0 with value: 1.571240971766251.


[LightGBM] [Info] Number of positive: 2622, number of negative: 3468
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.021783 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8984
[LightGBM] [Info] Number of data points in the train set: 6090, number of used features: 573
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.430542 -> initscore=-0.279641
[LightGBM] [Info] Start training from score -0.279641


[I 2024-06-28 11:28:11,071] Trial 1 finished with value: 1.5436638214051215 and parameters: {'learning_rate': 0.07150082148804313, 'num_leaves': 96, 'C': 0.19803005438106147}. Best is trial 0 with value: 1.571240971766251.


...

[I 2024-06-28 11:28:21,526] Trial 7 finished with value: 1.57715036112935 and parameters: {'learning_rate': 0.06731086196884159, 'num_leaves': 54, 'C': 1.8583687768146433}. Best is trial 3 with value: 1.5804333552199608.


[LightGBM] [Info] Number of positive: 2622, number of negative: 3468
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.035974 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8984
[LightGBM] [Info] Number of data points in the train set: 6090, number of used features: 573
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.430542 -> initscore=-0.279641
[LightGBM] [Info] Start training from score -0.279641


[I 2024-06-28 11:28:23,547] Trial 8 finished with value: 1.577806959947472 and parameters: {'learning_rate': 0.062352967829123994, 'num_leaves': 76, 'C': 7.175117429677378}. Best is trial 3 with value: 1.5804333552199608.


[LightGBM] [Info] Number of positive: 2622, number of negative: 3468
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.023966 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8984
[LightGBM] [Info] Number of data points in the train set: 6090, number of used features: 573
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.430542 -> initscore=-0.279641
[LightGBM] [Info] Start training from score -0.279641


[I 2024-06-28 11:28:24,213] Trial 9 finished with value: 1.5489166119500983 and parameters: {'learning_rate': 0.040027631946951144, 'num_leaves': 20, 'C': 0.3962151566968075}. Best is trial 3 with value: 1.5804333552199608.


In [23]:
# Get the best hyperparameters
best_params = study.best_params

In [25]:
lgb_model = LGBMClassifier(**best_params)
lgb_model.fit(X_train, y_train)


[LightGBM] [Warning] Unknown parameter: C
[LightGBM] [Warning] Unknown parameter: C
[LightGBM] [Info] Number of positive: 2622, number of negative: 3468
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.036950 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8984
[LightGBM] [Info] Number of data points in the train set: 6090, number of used features: 573
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.430542 -> initscore=-0.279641
[LightGBM] [Info] Start training from score -0.279641


LGBMClassifier(C=3.9456705222638484, learning_rate=0.07961950929353762,
               num_leaves=80)

In [26]:
lr_model = LogisticRegression(C=best_params['C'])
lr_model.fit(X_train, y_train)

LogisticRegression(C=3.9456705222638484)

In [27]:
# Evaluate models on test data
lgb_test_pred = lgb_model.predict(x_test)
lr_test_pred = lr_model.predict(x_test)


[LightGBM] [Warning] Unknown parameter: C


In [28]:
plot_optimization_history(study)

In [51]:
storage = 'sqlite:///study.db'

# Create a new study or load an existing one
study = optuna.create_study(study_name='example_study', storage=storage, load_if_exists=True)

# Now you can use the study for optimization
def objective(trial):
    x = trial.suggest_float('x', -10, 10)
    return (x - 2) ** 2

study.optimize(objective, n_trials=100)

[I 2024-06-28 12:21:02,470] A new study created in RDB with name: example_study
[I 2024-06-28 12:21:02,568] Trial 0 finished with value: 5.763994488935476 and parameters: {'x': -0.40083204096735514}. Best is trial 0 with value: 5.763994488935476.
[I 2024-06-28 12:21:02,640] Trial 1 finished with value: 8.201261162347977 and parameters: {'x': -0.863784412686817}. Best is trial 0 with value: 5.763994488935476.
[I 2024-06-28 12:21:02,705] Trial 2 finished with value: 116.65486041766607 and parameters: {'x': -8.800687960387805}. Best is trial 0 with value: 5.763994488935476.
[I 2024-06-28 12:21:02,769] Trial 3 finished with value: 24.21941967064944 and parameters: {'x': -2.921322959393078}. Best is trial 0 with value: 5.763994488935476.
[I 2024-06-28 12:21:02,833] Trial 4 finished with value: 64.85417735133412 and parameters: {'x': -6.05320913371397}. Best is trial 0 with value: 5.763994488935476.
[I 2024-06-28 12:21:02,899] Trial 5 finished with value: 8.26151234638698 and parameters: {'x

In [53]:
from sqlalchemy import create_engine
from optuna.storages import RDBStorage

# Create the database engine
engine = create_engine('sqlite:///study.db')

# Use the RDBStorage with Optuna
storage = RDBStorage(str(engine.url))

# Create a study
study = optuna.create_study(study_name='example_study', storage=storage, load_if_exists=True)

[I 2024-06-28 12:22:58,435] Using an existing study with name 'example_study' instead of creating a new one.


In [54]:

storage = optuna.storages.RDBStorage('sqlite:///study.db')
storage.upgrade()

In [55]:

!optuna-dashboard sqlite:///study.db

Listening on http://127.0.0.1:8080/
Hit Ctrl-C to quit.

Traceback (most recent call last):
  File "/usr/local/bin/optuna-dashboard", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/optuna_dashboard/_cli.py", line 140, in main
    run_wsgiref(app, args.host, args.port, args.quiet)
  File "/usr/local/lib/python3.10/dist-packages/optuna_dashboard/_cli.py", line 43, in run_wsgiref
    httpd = make_server(host, port, app, server_class=ThreadedWSGIServer)
  File "/usr/lib/python3.10/wsgiref/simple_server.py", line 154, in make_server
    server = server_class((host, port), handler_class)
  File "/usr/lib/python3.10/socketserver.py", line 452, in __init__
    self.server_bind()
  File "/usr/lib/python3.10/wsgiref/simple_server.py", line 50, in server_bind
    HTTPServer.server_bind(self)
  File "/usr/lib/python3.10/http/server.py", line 137, in server_bind
    socketserver.TCPServer.server_bind(self)
  File "/usr/lib/python3.10/socketserver.py", line 